### Kaggle ML course team battle 01 for baseline classification

#### 1. load data

In [1]:
import sklearn
from sklearn.datasets import load_files

# training
datapath = r'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/training_dataset_v1/training_dataset'
mrdataset = load_files(datapath, shuffle=True)

# testing
import pandas as pd
test = pd.read_csv('C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/test_dataset.csv', header=None)

# submission
sub = pd.read_csv('C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission.csv')

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

#### 2. baseline_model function

In [2]:
from sklearn.model_selection import cross_val_score

def baseline_model(model, vector, mrdataset, test , sub):
    performance = cross_val_score(model, tfidf_vectorizer.fit_transform(mrdataset.data), mrdataset.target, 
                                 cv=10, scoring='accuracy')
    print("avg-accuracy: {0}".format(performance.mean()))

    # test.head()
    x_train = tfidf_vectorizer.fit_transform(mrdataset.data)
    y_train = mrdataset.target
    x_test = tfidf_vectorizer.transform(test.iloc[:,1].values.astype('U'))

    model.fit(x_train,y_train)
    y_test = model.predict(x_test)

    sub['Label'] = y_test
    sub['Label'].replace([0,1],['neg','pos'], inplace=True)
    
    return sub
    

#### 3. Multinomial NB

In [3]:
# NB
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
NB = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_NB_ans.csv'
NB.to_csv(path, index=0)

avg-accuracy: 0.8647600000000001


#### 4. SVM

In [4]:
# SVM
from sklearn import svm
model = svm.SVC(kernel='linear')
svm = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_svm_ans.csv'
svm.to_csv(path, index=0)

avg-accuracy: 0.89564


#### 4-1. SVM 調參

In [12]:
# 參數調起來!!! 
from sklearn.model_selection import GridSearchCV
from sklearn import svm

x_train = tfidf_vectorizer.fit_transform(mrdataset.data)
y_train = mrdataset.target
x_test = tfidf_vectorizer.transform(test.iloc[:,1].values.astype('U'))

parameters = {'kernel':('linear', 'rbf'),'C':[1,2,4],'gamma':[0.125, 0.25, 0.5, 1, 2, 4]}
model = svm.SVC()
gs_clf = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf.fit(tfidf_vectorizer.fit_transform(tfidf_vectorizer.fit_transform(mrdataset.data)[:400]),
           mrdataset.target[:400])
performance = cross_val_score(gs_clf.best_estimator_, tfidf_vectorizer.fit_transform(mrdataset.data), mrdataset.target, 
                             cv=10, scoring='accuracy')
print("avg-accuracy: {0}".format(performance.mean()))
print('The parameters of the best model are :')
print(gs_clf.best_params_)


y_test = gs_clf.predict(x_test)

sub['Label'] = y_test
sub['Label'].replace([0,1],['neg','pos'], inplace=True)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_GS_ans.csv'
sub.to_csv(path, index=0)

AttributeError: lower not found

<400x74849 sparse matrix of type '<class 'numpy.float64'>'
	with 54858 stored elements in Compressed Sparse Row format>

#### 5. LR

In [5]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
LR = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_LR_ans.csv'
LR.to_csv(path, index=0)

C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


avg-accuracy: 0.88928


#### 6. RF

In [6]:
# RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
RF = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_RF_ans.csv'
RF.to_csv(path, index=0)

C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

avg-accuracy: 0.73864


C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


#### 7. kNN

In [7]:
# kNN
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
kNN = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_kNN_ans.csv'
kNN.to_csv(path, index=0)

avg-accuracy: 0.7749600000000001


#### 8. DT

In [8]:
# DT
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
DT = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_DT_ans.csv'
DT.to_csv(path, index=0)

avg-accuracy: 0.70404


#### 9. XGBoost

In [9]:
# XGboost
from xgboost import XGBClassifier
model = XGBClassifier()
XGB = baseline_model(model, tfidf_vectorizer, mrdataset, test , sub)

path = 'C:/Users/doudi/Downloads/tmu-ml-course-team-battle-1/submission_XGB_ans.csv'
XGB.to_csv(path, index=0)

avg-accuracy: 0.80848
